In [ ]:
!pip install transformers peft trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
import os
import torch
import gc
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoTokenizer
from transformers import TrainingArguments,pipeline
from peft import LoraConfig, PeftModel, get_peft_config
from trl import SFTTrainer

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
device_map = "auto"

In [ ]:
df = pd.read_csv("data/medquad.csv")

In [ ]:
df.shape

(16412, 4)

In [ ]:
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
data = Dataset.from_pandas(pd.DataFrame(data=df))

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
#"daryl149/llama-2-7b-chat-hf"
#model_name = "meta-llama/Llama-2-7b-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
model.config.pretraining_tp = 1
torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
LORA_ALPHA = 16
LORA_DROPOUT = 0.2
LORA_R = 64

In [ ]:
peft_config = LoraConfig(
        lora_alpha= LORA_ALPHA,
        lora_dropout= LORA_DROPOUT,
        r= LORA_R,
        bias="none",
        task_type="CAUSAL_LM",
    )

In [ ]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10
BATCH_SIZE = 16
WEIGHT_DECAY = 0.001
MAX_GRAD_NORM = 0.3
gradient_accumulation_steps = 16
STEPS = 1
OPTIM = "adam"
MAX_STEPS = 10

In [ ]:
OUTPUT_DIR = "./results"

In [ ]:
training_args = TrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps= gradient_accumulation_steps,
    learning_rate= LEARNING_RATE,
    logging_steps= STEPS,
    num_train_epochs= NUM_EPOCHS,
    max_steps= MAX_STEPS,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="question",
    max_seq_length=500,
    tokenizer=tokenizer,
    args=training_args,
)

Map:   0%|          | 0/16412 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
1,4.001800
2,3.930400
3,4.021600
4,3.873800
5,3.849200
6,3.808600
7,3.831600
8,3.814700
9,3.741900
10,3.919100


TrainOutput(global_step=10, training_loss=3.879258561134338, metrics={'train_runtime': 939.7707, 'train_samples_per_second': 2.724, 'train_steps_per_second': 0.011, 'total_flos': 1401121747107840.0, 'train_loss': 3.879258561134338, 'epoch': 0.16})

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pipe = pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_length=500)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
prompt = "What is Glaucoma?"

In [ ]:
template = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest Medical assistant.
You always answer to the context and do not hallucinate. You only answer to the questions that are related to Medical and Healthcare.
If a question does not make any sense,Just answer I don't know, please don't share false information.
<</SYS>>
{prompt} [/INST]
"""

In [ ]:
result = pipe(template)

In [ ]:
response = result[0]['generated_text']
response

"<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest Medical assistant. \nYou always answer to the context and do not hallucinate. You only answer to the questions that are related to Medical and Healthcare. \nIf a question does not make any sense,Just answer I don't know, please don't share false information.\n<</SYS>>\nWhat is Glaucoma? [/INST]\nHello! I'm here to help you with any questions you may have about medical and healthcare. Glaucoma is a group of eye conditions that can damage the optic nerve, which carries visual information from the eye to the brain. It is often associated with increased pressure inside the eye, which can lead to vision loss and even blindness if left untreated.\n\nThere are several types of glaucoma, including:\n\n1. Open-angle glaucoma: This is the most common form of glaucoma, and it occurs when the iris and the trabecular meshwork in the eye are not functioning properly, allowing fluid to build up and increase pressure inside the eye.\n2. Clos